In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv("../data/train.csv", index_col=0)

In [3]:
df['numbers'] = df.Sequence.str.split(',').map(np.float128)

In [4]:
from keras.preprocessing.sequence import pad_sequences

def truncate_numbers(numbers, maxlen=100, normalize=True):
    """
    Truncate sequences to have max length of <maxlen>
    and split into X, y
    """
    trunc = pad_sequences(numbers, maxlen, dtype=np.float128)
    X = trunc[:, :-1]
    X = X.reshape(X.shape[0], X.shape[1], 1)
    y = trunc[:, -1]
    return X, y

Using TensorFlow backend.
/Users/ptyshevs/envs/loc_env/lib/python3.6/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.24) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [5]:
trunc_X, trunc_y = truncate_numbers(df.numbers[:10000], maxlen=20)
X_mean, X_std = trunc_X.mean(), trunc_X.std()
y_mean, y_std = trunc_y.mean(), trunc_y.std()
trunc_X = (trunc_X - X_mean) / X_std
trunc_y = (trunc_y - y_mean) / y_std

In [6]:
from keras.models import Sequential
from keras.layers import LSTM, Dense
model = Sequential()
model.add(LSTM(5, input_shape=(19, 1), return_sequences=True))
model.add(LSTM(5))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 19, 5)             140       
_________________________________________________________________
lstm_2 (LSTM)                (None, 5)                 220       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 6         
Total params: 366
Trainable params: 366
Non-trainable params: 0
_________________________________________________________________


In [7]:
history = model.fit(trunc_X, trunc_y, batch_size=100, verbose=1)

Epoch 1/1
10000/10000 [==============================] - 3s 332us/step - loss: 1.0002


In [8]:
# non-sense
model.predict(trunc_X[:5]).astype(np.float128) * y_std + y_mean

array([[9.82886497e+182],
       [9.82886497e+182],
       [9.82886497e+182],
       [9.82886497e+182],
       [9.82886497e+182]], dtype=float128)